In [20]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.wrappers import LdaMallet
from gensim.test.utils import common_corpus, common_dictionary

# spacy for lemmatization
import spacy

import en_core_web_sm
nlp = en_core_web_sm.load()

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [21]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [52]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/Brix-mix/jsonpapers/master/regte.json')
print(df.title.unique())
df.head()

['A formal qualitative risk management approach for IT security'
 'A Framework of Opportunity-Reducing Techniques to Mitigate the Insider Threat-Towards Best Practice'
 'Cloud Supply Chain Resilience- A Coordination Approach'
 'Data Aggregation Using Homomorphic Encryption in Wireless Sensor Networks'
 'Evaluation and Analysis of a Software Prototype for Guidance and Implementation of a Standardized Digital Forensic Investigation Process'
 'Intrusion Detection in Bluetooth Enabled Mobile Phones'
 'Location Aware Mobile Device Management'
 'Mapping ‘Security Safeguard’ Requirements in a Data Privacy legislation to an International Privacy Framework- A Compliance Methodology'
 'Playing Hide-and-Seek- Detecting the Manipulation of Android Timestamps'
 'Prerequisites for building a security incident response capability'
 'Protection of personal information in the South African Cloud Computing environment- A framework for Cloud Computing adoption'
 'Risk-Driven Security Metrics Development 

,abstracts,title
0,"Information technology (IT) security, which is...",A formal qualitative risk management approach ...
1,This paper presents a unified framework derive...,A Framework of Opportunity-Reducing Techniques...
10,Cloud computing is a service-based computing r...,Cloud Supply Chain Resilience- A Coordination ...
11,Secure information aggregation using homomorph...,Data Aggregation Using Homomorphic Encryption ...
12,Performing a digital forensic investigation re...,Evaluation and Analysis of a Software Prototyp...


In [53]:
# Convert to list
data = df.abstracts.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Information technology (IT) security, which is concerned about protecting '
 'the confidentiality, integrity and availability of information technology '
 'assets, inherently possesses a significant amount of risk, some known and '
 'some unknown. IT security risk management has gained considerable attention '
 'over the past decade due to the collapsing of some large organisations in '
 'the world. Previous investigative research in the field of IT security have '
 'indicated that despite the efforts that organisations employ to reduce IT '
 'security risks, the trend of IT security attacks are still increasing. One '
 'of the contributing factors to poor management of IT security risk is '
 'attributed to the fact that IT security risk management is often left to the '
 'technical security technologist who do not necessarily employ formal risk '
 'management tools and reasoning. For this reason, organisations find '
 'themselves in a position where they do not have the correct appr

In [54]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['information', 'technology', 'it', 'security', 'which', 'is', 'concerned', 'about', 'protecting', 'the', 'confidentiality', 'integrity', 'and', 'availability', 'of', 'information', 'technology', 'assets', 'inherently', 'possesses', 'significant', 'amount', 'of', 'risk', 'some', 'known', 'and', 'some', 'unknown', 'it', 'security', 'risk', 'management', 'has', 'gained', 'considerable', 'attention', 'over', 'the', 'past', 'decade', 'due', 'to', 'the', 'collapsing', 'of', 'some', 'large', 'organisations', 'in', 'the', 'world', 'previous', 'investigative', 'research', 'in', 'the', 'field', 'of', 'it', 'security', 'have', 'indicated', 'that', 'despite', 'the', 'efforts', 'that', 'organisations', 'employ', 'to', 'reduce', 'it', 'security', 'risks', 'the', 'trend', 'of', 'it', 'security', 'attacks', 'are', 'still', 'increasing', 'one', 'of', 'the', 'contributing', 'factors', 'to', 'poor', 'management', 'of', 'it', 'security', 'risk', 'is', 'attributed', 'to', 'the', 'fact', 'that', 'it', 'se

In [55]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

C:\Users\s211144843\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['information', 'technology', 'it', 'security', 'which', 'is', 'concerned', 'about', 'protecting', 'the', 'confidentiality', 'integrity', 'and', 'availability', 'of', 'information', 'technology', 'assets', 'inherently', 'possesses', 'significant', 'amount', 'of', 'risk', 'some', 'known', 'and', 'some', 'unknown', 'it', 'security', 'risk', 'management', 'has', 'gained', 'considerable', 'attention', 'over', 'the', 'past', 'decade', 'due', 'to', 'the', 'collapsing', 'of', 'some', 'large', 'organisations', 'in', 'the', 'world', 'previous', 'investigative', 'research', 'in', 'the', 'field', 'of', 'it', 'security', 'have', 'indicated', 'that', 'despite', 'the', 'efforts', 'that', 'organisations', 'employ', 'to', 'reduce', 'it', 'security', 'risks', 'the', 'trend', 'of', 'it', 'security', 'attacks', 'are', 'still', 'increasing', 'one', 'of', 'the', 'contributing', 'factors', 'to', 'poor', 'management', 'of', 'it', 'security', 'risk', 'is', 'attributed', 'to', 'the', 'fact', 'that', 'it', 'sec

In [56]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [57]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['information', 'technology', 'security', 'concern', 'protect', 'confidentiality', 'integrity', 'availability', 'information', 'technology', 'asset', 'inherently', 'possess', 'significant', 'amount', 'risk', 'know', 'unknown', 'security', 'risk', 'management', 'gain', 'considerable', 'attention', 'past', 'decade', 'due', 'collapse', 'large', 'organisation', 'world', 'previous', 'investigative', 'research', 'field', 'security', 'indicate', 'effort', 'organisation', 'employ', 'reduce', 'security', 'risk', 'trend', 'security', 'attack', 'still', 'increase', 'contributing', 'factor', 'poor', 'management', 'security', 'risk', 'attribute', 'fact', 'security', 'risk', 'management', 'often', 'leave', 'technical', 'security', 'technologist', 'necessarily', 'employ', 'formal', 'risk', 'management', 'tool', 'reasoning', 'reason', 'organisation', 'find', 'position', 'correct', 'approach', 'identify', 'assess', 'treat', 'security', 'risk', 'employ', 'formal', 'risk', 'base', 'approach', 'manage', 

In [58]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 6), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 3), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 2), (60, 5), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 5), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 2), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 3), (83, 1), (84, 17), (85, 1), (86, 16), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 3), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1)]]


In [59]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('account', 1),
  ('aim', 1),
  ('amount', 1),
  ('analysis', 2),
  ('approach', 6),
  ('assess', 2),
  ('assessment', 1),
  ('asset', 1),
  ('assist', 1),
  ('attack', 1),
  ('attention', 2),
  ('attribute', 1),
  ('availability', 1),
  ('available', 1),
  ('base', 1),
  ('basis', 1),
  ('better', 1),
  ('cater', 1),
  ('characteristic', 1),
  ('coherent', 1),
  ('collapse', 1),
  ('complex', 1),
  ('comprehensive', 1),
  ('concern', 1),
  ('confidentiality', 1),
  ('considerable', 1),
  ('consolidated', 1),
  ('contribute', 1),
  ('contributing', 1),
  ('correct', 2),
  ('decade', 1),
  ('define', 1),
  ('due', 1),
  ('dynamic', 1),
  ('effort', 1),
  ('employ', 3),
  ('enable', 1),
  ('ensure', 1),
  ('fact', 1),
  ('factor', 1),
  ('field', 2),
  ('find', 1),
  ('formal', 2),
  ('framework', 1),
  ('fundamental', 1),
  ('gain', 1),
  ('identify', 2),
  ('incorporate', 1),
  ('increase', 1),
  ('indicate', 1),
  ('information', 2),
  ('inherently', 1),
  ('integrity', 1),
  ('inve

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [63]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.045*"opportunity" + 0.026*"theory" + 0.023*"reduce" + 0.023*"framework" + '
  '0.019*"paper" + 0.015*"technique" + 0.015*"insider" + 0.015*"mitigate" + '
  '0.015*"threat" + 0.015*"derive"'),
 (1,
  '0.073*"forensic" + 0.052*"research" + 0.040*"database" + 0.034*"material" + '
  '0.026*"cloud" + 0.021*"area" + 0.021*"scienti" + 0.020*"new" + '
  '0.014*"survey" + 0.014*"end"'),
 (2,
  '0.026*"web" + 0.022*"beacon" + 0.016*"smart" + 0.013*"resource" + '
  '0.013*"space" + 0.010*"analogue" + 0.010*"rfid" + 0.010*"research" + '
  '0.010*"investigation" + 0.010*"scanning"'),
 (3,
  '0.001*"forensic" + 0.001*"information" + 0.001*"cloud" + 0.001*"security" + '
  '0.001*"popi" + 0.001*"digital" + 0.001*"privacy" + 0.001*"personal" + '
  '0.001*"practitioner" + 0.001*"south_african"'),
 (4,
  '0.044*"surveillance" + 0.029*"national" + 0.025*"state" + '
  '0.025*"government" + 0.025*"communication" + 0.020*"agency" + '
  '0.020*"security" + 0.020*"conduct" + 0.020*"purpose" + 0.015*"

In [61]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.874256318292994

Coherence Score:  0.4923410391123399


In [62]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      11.474788        1       1  0.173984  0.080791
6      10.439507        1       2 -0.053886  0.182926
16     10.199450        1       3  0.045477  0.048039
19      8.768431        1       4  0.088279  0.025196
14      8.611834        1       5 -0.110576  0.198885
18      7.311854        1       6  0.018671 -0.018228
2       7.023938        1       7  0.063305  0.017639
17      6.781790        1       8  0.067726 -0.055331
0       5.626378        1       9 -0.015352 -0.090696
7       4.783302        1      10  0.090799 -0.024264
10      4.111297        1      11 -0.082713 -0.008912
4       3.973582        1      12 -0.179986 -0.059487
13      3.385532        1      13  0.039257 -0.048738
11      3.017871        1      14 -0.094815 -0.066666
1       2.424520        1      15  0.004509 -0.046353
15      1.990317        1      16 -0.017285 -0.033770
3       0.019619        1      17 -0.010374 -0.022424
5       0.018761        1      18 -0.007575 -0.026820
12      0.018647        1      19 -0.009739 -0.027038
8       0.018592        1      20 -0.009707 -0.024750, topic_info=     Category       Freq                Term      Total  loglift  logprob
term                                                                     
256   Default  26.000000            forensic  26.000000  30.0000  30.0000
86    Default  44.000000            security  44.000000  29.0000  29.0000
153   Default  19.000000               cloud  19.000000  28.0000  28.0000
670   Default  16.000000             website  16.000000  27.0000  27.0000
82    Default  18.000000            research  18.000000  26.0000  26.0000
84    Default  20.000000                risk  20.000000  25.0000  25.0000
125   Default  10.000000         opportunity  10.000000  24.0000  24.0000
50    Default  34.000000         information  34.000000  23.0000  23.0000
779   Default   9.000000        surveillance   9.000000  22.0000  22.0000
403   Default  14.000000                popi  14.000000  21.0000  21.0000
95    Default  10.000000                tool  10.000000  20.0000  20.0000
245   Default  26.000000             digital  26.000000  19.0000  19.0000
682   Default  10.000000               image  10.000000  18.0000  18.0000
850   Default   7.000000             science   7.000000  17.0000  17.0000
424   Default   7.000000             android   7.000000  16.0000  16.0000
405   Default  14.000000             privacy  14.000000  15.0000  15.0000
440   Default   7.000000            database   7.000000  14.0000  14.0000
912   Default  11.000000                 web  11.000000  13.0000  13.0000
324   Default  11.000000              device  11.000000  12.0000  12.0000
446   Default   5.000000               event   5.000000  11.0000  11.0000
592   Default   6.000000  forensic_readiness   6.000000  10.0000  10.0000
73    Default  23.000000             process  23.000000   9.0000   9.0000
232   Default  14.000000              system  14.000000   8.0000   8.0000
209   Default   8.000000          encryption   8.000000   7.0000   7.0000
402   Default  10.000000            personal  10.000000   6.0000   6.0000
1133  Default   6.000000         transaction   6.000000   5.0000   5.0000
288   Default   8.000000           prototype   8.000000   4.0000   4.0000
74    Default  14.000000             propose  14.000000   3.0000   3.0000
43    Default  14.000000           framework  14.000000   2.0000   2.0000
589   Default   5.000000                 esc   5.000000   1.0000   1.0000
...       ...        ...                 ...        ...      ...      ...
1012  Topic20   0.000544          illustrate   1.403313   0.7340  -7.1100
859   Topic20   0.000571          complexity   2.307778   0.2855  -7.0610
860   Topic20   0.000571         consequence   2.307229   0.2855  -7.0613
881   Topic20   0.000570           regulator   2.307901   0.2832  -7.0633
1017  Topic20   0.000543     pe

In [65]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'C:/yes/mallet/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\S21114~1\\AppData\\Local\\Temp\\1fe93d_state.mallet.gz'

In [47]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [48]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

CalledProcessError: Command 'C:\yes\mallet-2.0.8in\mallet import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\S21114~1\AppData\Local\Temp\447fa9_corpus.txt --output C:\Users\S21114~1\AppData\Local\Temp\447fa9_corpus.mallet' returned non-zero exit status 1.